In [2]:
import csv
import numpy as np
import os
import glob
import subprocess
import pandas as pd

In [ ]:
# write ligands seperately

mols = Chem.SDMolSupplier(
    "/home/anna/Documents/benchmark/inputs/industrybenchmarks2024/p38/schroedinger22jacs_ligands.sdf"
)
for mol in mols:
    # print(mol.GetProp("_Name"))
    with Chem.SDWriter(
        f'/home/anna/Documents/benchmark/inputs/industrybenchmarks2024/p38/ligands_schroedinger22jacs_intermediates/lig_{mol.GetProp("_Name").replace("p38a_","").replace("_","")}.sdf'
    ) as w:
        w.write(mol)

# rename
name_dict = {}
for prot in ["syk", "hif2a", "cmet"]:
    jacs_file = f"/home/anna/Documents/benchmark/inputs/{prot}/results_edges_5ns.csv"
    mols = Chem.SDMolSupplier(
        f"/home/anna/Documents/benchmark/inputs/{prot}/ligands.sdf"
    )
    name_dict[prot] = {}
    for mol, idx in zip(mols, range(1, len(mols) + 1, 1)):
        name_dict[prot][mol.GetProp("_Name")] = f"lig_{idx}"

In [ ]:
# professa results

# file = f"/home/anna/Documents/benchmark/inputs/other_computed/professa/professa_{prot}_results.dat"

# df = pd.read_csv(file, delimiter=",")

# perts_dict = {}
# for index, row in df.iterrows():
#     perts_dict[f"{row['perturbation']}"] = (float(row["ddG"]), float(row["ddG_error"]))
# write_perts_file(
#     perts_dict,
#     # .csv
#     file_path=f"/home/anna/Documents/benchmark/inputs/{prot}/perts_file_professa",
# )

In [ ]:
# plotting stats distributions
results_dict = net_proc_dict

# make best fit dict
for protein in ["tyk2", "mcl1"]:  # , "p38"
    print(protein)
    best_fit_dict = {}

    for network in ["lomap", "rbfenn"]:
        for eng in ana_obj.engines:
            mu, std, ci = results_dict[network][protein][eng]

            try:
                # Plot the PDF.
                if stats == "spearmans_coeff":
                    xmin, xmax = plt.xlim(left=-1, right=1)
                elif stats == "MAE":
                    xmin, xmax = plt.xlim(left=0, right=mu + 1)
                x = np.linspace(xmin, xmax, 100)
                y = norm.pdf(x, mu, std)

                best_fit_dict[f"{network}_{eng}"] = ((x, y), mu, std)

            except:
                pass

    col_dict = {}

    for key in best_fit_dict.keys():
        hexadecimal_alphabets = "0123456789ABCDEF"
        color = "#" + "".join([random.choice(hexadecimal_alphabets) for j in range(6)])
        col_dict[key] = color

    # plot the distributions
    fig, ax = plt.subplots(figsize=(10, 10))

    lines = []
    mu_std_string = ""

    for name in best_fit_dict.keys():
        col = col_dict[name]
        plt.plot(
            best_fit_dict[name][0][0],
            best_fit_dict[name][0][1],
            "k",
            linewidth=2,
            color=col,
        )
        lines += plt.plot(0, 0, c=col, label=name)
        mu_std_string += f"\n{name} : mu = {best_fit_dict[name][1]:.3f} , std = {best_fit_dict[name][2]:.3f}"

        ax.axvspan(
            best_fit_dict[name][1] + best_fit_dict[name][2],
            best_fit_dict[name][1] - best_fit_dict[name][2],
            color=col,
            alpha=0.2,
        )
    labels = [l.get_label() for l in lines]
    plt.legend(lines, labels, loc="upper right")

    plt.xlabel("Error")
    plt.ylabel("Frequency")
    plt.title(f"{protein}, {stats}\n{mu_std_string}")
    plt.savefig(
        f"/backup/overall_analysis/{protein}_rbfenn_lomap_{stats}_distribution_from_bootstrapping.png",
        dpi=300,
        bbox_inches="tight",
    )
    plt.show()

**RERUNS ANALYSIS**

In [ ]:
# rerun analysis

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

# import libraries
from pipeline.analysis import *
from pipeline.utils import *
from pipeline import *
import logging
from scipy.stats import sem as sem

logging.getLogger().setLevel(logging.INFO)

print(BSS.__file__)

In [ ]:
def flatten_comprehension(matrix):
    return [item for row in matrix for item in row]

In [ ]:
protein = "p38"
bench_folder = f"/home/anna/Documents/benchmark"
main_dir = f"{bench_folder}/reruns/{protein}"
engine = "GROMACS"

ana_prot = analysis_protocol(
    {
        "estimator": "MBAR",
        "method": "alchemlyb",
        "check overlap": True,
        "try pickle": True,
        "save pickle": True,
        "auto equilibration": False,
        "statistical inefficiency": False,
        "truncate lower": 0,
        "truncate upper": 100,
        "name": None,
    }
)

net_file = f"{main_dir}/execution_model/network_combined.dat"
exp_file = f"{bench_folder}/inputs/experimental/{protein}.yml"
output_folder = f"{main_dir}/outputs_extracted"

pipeline_prot = pipeline_protocol(
    f"{main_dir}/execution_model/protocol.dat", auto_validate=True
)  # no protocol used , name added after if needed
# pipeline_prot = pipeline_protocol(auto_validate=True)

# initialise the network object
ana_obj = analysis_network(
    output_folder,
    exp_file=exp_file,
    net_file=net_file,
    analysis_prot=ana_prot,
    # method=pipeline_prot.name(),  # if the protocol had a name
    engines=[engine],
)

# compute
ana_obj.compute_results()

# # add ligands folder
if os.path.isdir(f"{bench_folder}/inputs/reruns/{protein}/ligands_intermediates"):
    ana_obj.add_ligands_folder(
        f"{bench_folder}/inputs/reruns/{protein}/ligands_intermediates"
    )

In [ ]:
print(ana_obj.method)
perts = ana_obj.failed_perturbations("AMBER")
print(len(perts))
with open(net_file.replace(".dat", "_reverse.dat"), "w") as f:
    for pert in perts:
        print(pert)
        string = f"{pert.split('~')[1]} {pert.split('~')[0]} 12 0.0000,0.0909,0.1818,0.2727,0.3636,0.4546,0.5454,0.6364,0.7273,0.8182,0.9091,1.0000 {engine}\n"
        f.write(string)

In [ ]:
print(ana_obj.method)
perts = ana_obj.failed_perturbations("AMBER")
print(len(perts))
with open(net_file.replace(".dat", "_2fs.dat"), "w") as f:
    for pert in perts:
        print(pert)
        string = f"{pert.split('~')[0]} {pert.split('~')[1]} 12 0.0000,0.0909,0.1818,0.2727,0.3636,0.4546,0.5454,0.6364,0.7273,0.8182,0.9091,1.0000 {engine}\n"
        f.write(string)

# with open(net_file.replace(".dat", "_4fs.dat"), "w") as f:
#     for pert in ana_obj.perturbations:
#         if pert not in perts:
#             # print(pert)
#             string = f"{pert.split('~')[0]} {pert.split('~')[1]} 12 0.0000,0.0909,0.1818,0.2727,0.3636,0.4546,0.5454,0.6364,0.7273,0.8182,0.9091,1.0000 {engine}\n"
#             f.write(string)

# ana_obj.calc_rmse_engines("pert", recalculate=True)

In [ ]:
# overwrite calc pert dict
for prot in ana_obj_dict:
    ana_obj = network_dict["lomap"][prot]["subsampling"]

    calc_pert_dict = {}
    for eng in ana_obj.engines:
        calc_pert_dict[eng] = {}
        for pert in ana_obj._perturbations_dict[eng]:
            vals_list = []
            bound_vals_ = []
            free_vals_ = []
            try:
                for r in [0, 1, 2]:
                    bound_vals_.append(ana_obj.calc_repeat_bound_dict[eng][r][pert][0])
                    free_vals_.append(ana_obj.calc_repeat_free_dict[eng][r][pert][0])
                bound_vals = [x for x in bound_vals_ if str(x) != "nan"]
                free_vals = [x for x in free_vals_ if str(x) != "nan"]
                # print(bound_vals, free_vals)
                if len(bound_vals) > 2:
                    max_bound = (
                        [abs(val) for val in bound_vals / np.mean(bound_vals)]
                    ).index(max([abs(val) for val in bound_vals / np.mean(bound_vals)]))
                    del bound_vals[max_bound]
                if len(free_vals) > 2:
                    max_free = (
                        [abs(val) for val in free_vals / np.mean(free_vals)]
                    ).index(max([abs(val) for val in free_vals / np.mean(free_vals)]))
                    del free_vals[max_free]
                free_avg = np.mean(free_vals)
                bound_avg = np.mean(bound_vals)
                if len(free_vals) == 1:
                    free_err = ana_obj.calc_repeat_free_dict[eng][r][pert][1]
                else:
                    free_err = sem(free_vals)
                if len(bound_vals) == 1:
                    bound_err = ana_obj.calc_repeat_bound_dict[eng][r][pert][1]
                else:
                    bound_err = sem(bound_vals)
                freenrg_val = bound_avg - free_avg
                freenrg_err = math.sqrt(math.pow(bound_err, 2) + math.pow(free_err, 2))
                freenrg_rel = (freenrg_val, freenrg_err)
            except Exception as e:
                freenrg_rel = (np.nan, np.nan)
                print(e)
                print(f"{prot} {eng} {pert} does not have a value")

            calc_pert_dict[eng][pert] = freenrg_rel

    ana_obj.calc_pert_dict = calc_pert_dict
    # ana_obj._initialise_stats_object(check=False)

In [ ]:
# remove any pert greater than a certain value
for network in network_dict:
    ana_obj_dict = network_dict[network]
    for prot in ana_obj_dict:
        for name in ana_dicts:
            print(prot, name)
            ana_obj = ana_obj_dict[prot][name]

            try:
                for eng in ana_obj.engines:
                    perts = []
                    for pert in ana_obj.calc_pert_dict[eng]:
                        if abs(ana_obj.calc_pert_dict[eng][pert][0]) > 10:
                            perts.append(pert)
                    ana_obj.remove_perturbations(perts, name=eng)
                    print(f"{perts} removed for {network}, {prot}, {name}, {eng}")
            except:
                pass

In [ ]:
# remove any pert with error greater than a certain value
# for network in network_dict:
network = "lomap"
ana_obj_dict = network_dict[network]
for prot in ana_obj_dict:
    for name in ana_dicts:
        print(prot, name)
        ana_obj = ana_obj_dict[prot][name]

        try:
            for eng in ana_obj.engines:
                perts = []
                for pert in ana_obj.calc_pert_dict[eng]:
                    if abs(ana_obj.calc_pert_dict[eng][pert][1]) > 1:
                        perts.append(pert)
                ana_obj.remove_perturbations(perts, name=eng)
                print(f"{perts} removed for {network}, {prot}, {name}, {eng}")
        except:
            pass

In [ ]:
# # consensus scoring 2
# # picking the most similar from a euclidian similarity matrix

# from scipy.spatial.distance import euclidean, pdist, squareform


# def similarity_func(u, v):
#     return 1 / (1 + euclidean(u, v))


# for prot in ana_obj_dict.keys():
#     print(prot)
#     ana_obj = network_dict["lomap"][prot]["plain"]
#     pipeline.utils.validate.folder_path(
#         f"{ana_obj.files_folder}/consensus2", create=True
#     )

#     pert_dict = {}
#     for eng in ana_obj.engines:
#         pert_dict[eng] = []

#     for pert in ana_obj.perturbations:
#         pert_list = []
#         eng_list = []
#         for eng in ana_obj.engines:
#             try:
#                 pert_list.append(ana_obj.calc_pert_dict[eng][pert][0])
#                 eng_list.append(eng)
#             except:
#                 pass

#         DF_var = pd.DataFrame.from_dict({pert: pert_list})
#         DF_var.index = eng_list
#         if len(eng_list) < 3:
#             for eng in DF_var.index:
#                 pert_dict[eng].append(pert)

#         else:
#             try:
#                 dists = pdist(DF_var, similarity_func)
#                 DF_euclid = squareform(dists)
#                 # 1,2 is SOMD,GROMACS
#                 # 0,1 is AMBER,SOMD
#                 # 0,2 is AMBER, GROMACS
#                 # pick the most similar
#                 max_dict = {
#                     DF_euclid[1][2]: ["SOMD", "GROMACS"],
#                     DF_euclid[0][1]: ["AMBER", "SOMD"],
#                     DF_euclid[0][2]: ["AMBER", "GROMACS"],
#                 }
#                 maximum_sim = max(max_dict.keys())
#                 for eng in max_dict[maximum_sim]:
#                     pert_dict[eng].append(pert)
#             except:
#                 # if val is missing, use the other two engines
#                 DF_var.dropna(inplace=True)
#                 for eng in DF_var.index:
#                     pert_dict[eng].append(pert)

#     mod_results_files_list = []

#     for eng in ana_obj.engines:
#         mod_results_files = write_modified_results_files(
#             results_files=ana_obj._results_repeat_files[eng],
#             perturbations=pert_dict[eng],
#             name="consensus2",
#             output_folder=f"{ana_obj.files_folder}/consensus2",
#         )
#         mod_results_files_list.append(mod_results_files)

#     mod_results_files_list = flatten_comprehension(mod_results_files_list)

#     try:
#         ana_obj.compute_other_results(mod_results_files_list, name="consensus2")
#     except:
#         print(f"could not for {prot}")